# An inferential problem: The Gender Wage Gap

## Data analysis

In [ ]:
import pandas as pd
import numpy as np
import pyreadr as rr # package to use data form R format
import math

In [ ]:
#!pip install pyreadr==0.4.2

In [ ]:
#rdata_read = pyreadr.read_r("../../data/wage2015_subsample_inference.Rdata")

data  = pd.read_csv(r'C:/Users/pamelacubas/Downloads/wage2015_subsample_inference.csv')

# Extracting the data frame from rdata_read
#data = rdata_read[ 'data' ]
data['occ']=pd.Categorical(data.occ)
data['occ2']=pd.Categorical(data.occ2)
data['ind']=pd.Categorical(data.ind)
data['ind2']=pd.Categorical(data.ind2)

data

In [ ]:
data.shape

In [ ]:
data.info()

***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In this case, we Focus on the subset of college-advanced-educated workers. he analysis should be analogous to what we’ve presented – explaining basic, control and partialling out model, generating point estimates and standard errors.

In [ ]:
data = data[(data.scl == 1) | (data.ad == 1) | (data.clg == 1)]
data

In [ ]:
data.shape

To start our (causal) analysis, we compare the sample means given gender:

In [ ]:
Z = data[ ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] ]

data_female = data[data[ 'sex' ] == 1 ]
Z_female = data_female[ ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] ]

data_male = data[ data[ 'sex' ] == 0 ]
Z_male = data_male[ [ "lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1" ] ]


table = np.zeros( (12, 3) )
table[:, 0] = Z.mean().values
table[:, 1] = Z_male.mean().values
table[:, 2] = Z_female.mean().values
table_pandas = pd.DataFrame( table, columns = [ 'All', 'Men', 'Women']) # from table to dataframe
table_pandas.index = ["Log Wage","Sex","Some High School","High School Graduate","Some College","Gollage Graduate","Advanced Degree", "Northeast","Midwest","South","West","Experience"]
table_html = table_pandas.to_html() # html format

table_pandas

In [ ]:
print( table_html )

In particular, the table above shows that the difference in average logwage between men and women is equal to  0,075

In [ ]:
data_female['lwage'].mean()- data_male['lwage'].mean()

Thus, the unconditional gender wage gap is about 7.5% for the group of never married workers (women get paid less on average in our sample). We also observe that never married working women are relatively more educated than working men and have lower working experience.

In [ ]:
rdata_read = rr.read_r(r"../../../data/wage2015_subsample_inference.Rdata")

xx
# Extracting the data frame from rdata_read
data = rdata_read[ 'data' ]


data.shape

This unconditional (predictive) effect of gender equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}

We verify this by running an ols regression in R.

In [ ]:
pip install statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
nocontrol_model = smf.ols( formula = 'lwage ~ sex', data = data )
nocontrol_est = nocontrol_model.fit().summary2().tables[1]['Coef.']['sex']
nocontrol_est
nocontrol_se2 = nocontrol_model.fit().summary2().tables[1]['Std.Err.']['sex']


# robust standar erros
HCV_coefs = nocontrol_model.fit().cov_HC0
nocontrol_se = np.power( HCV_coefs.diagonal() , 0.5)[1]
nocontrol_se

# print unconditional effect of gender and the corresponding standard error

print( f'The estimated gender coefficient is {nocontrol_est} and the corresponding standard error is {nocontrol_se2}' )
print( f'The estimated gender coefficient is {nocontrol_est} and the corresponding robust standard error is {nocontrol_se}','\n' )


# confidence interval
nocontrol_model.fit().conf_int( alpha=0.05 ).loc[['sex']]


Note that the standard error is computed with the *R* package *sandwich* to be robust to heteroskedasticity. 


Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

Let us run the ols regression with controls.

## Ols regression with controls

In [ ]:
flex = 'lwage ~ sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'

# The smf api replicates R script when it transform data
control_model = smf.ols( formula = flex, data = data )
control_est = control_model.fit().summary2().tables[1]['Coef.']['sex']

print(control_model.fit().summary2().tables[1])

HCV_coefs = control_model.fit().cov_HC0
control_se = np.power( HCV_coefs.diagonal() , 0.5)[42]  # error standard for sex's coefficients 

control_se

print( f"Coefficient for OLS with controls {control_est} and the corresponding robust standard error is {control_se}" )

# confidence interval
control_model.fit().conf_int( alpha=0.05 ).loc[['sex']]


In [ ]:
control_model 

The estimated regression coefficient  𝛽1≈−0.0676 measures how our linear prediction of wage changes if we set the gender variable  𝐷 from 0 to 1, holding the controls  𝑊 fixed. We can call this the predictive effect (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size  8 % for women decreases to about  7
 % after controlling for worker characteristics.


Next, we are using the Frisch-Waugh-Lovell theorem from the lecture partialling-out the linear effect of the controls via ols.

## Partialling-Out using ols

In [ ]:
# models
# model for Y
flex_y = 'lwage ~  (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'
# model for D
flex_d = 'sex ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)' 

# partialling-out the linear effect of W from Y
t_Y = smf.ols( formula = flex_y , data = data ).fit().resid

# partialling-out the linear effect of W from D
t_D = smf.ols( formula = flex_d , data = data ).fit().resid


data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )


# regression of Y on D after partialling-out the effect of W
partial_fit =  smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_est = partial_fit.summary2().tables[1]['Coef.']['t_D']


# standard error
HCV_coefs = partial_fit.cov_HC0
partial_se = np.power( HCV_coefs.diagonal() , 0.5)[1]

print( f"Coefficient for D via partialling-out {partial_est} and the corresponding robust standard error is {partial_se}" )

# confidence interval
partial_fit.conf_int( alpha=0.05 ).loc[['t_D']]


In [ ]:
#np.vstack(( t_Y.values , t_D.values )).T

data_res = pd.DataFrame( np.vstack(( t_Y.values , t_D.values )).T , columns = [ 't_Y', 't_D' ] )
data_res

Again, the estimated coefficient measures the linear predictive effect (PE) of $D$ on $Y$ after taking out the linear effect of $W$ on both of these variables. This coefficient equals the estimated coefficient from the ols regression with controls.

We know that the partialling-out approach works well when the dimension of $W$ is low
in relation to the sample size $n$. When the dimension of $W$ is relatively high, we need to use variable selection
or penalization for regularization purposes. 


Use appropiate plots (i.e hist, barplots, scatter plots , pie plots, etc) to describe main varaibles (wage, log-wage, sex, some college, college graduate, avdanced degree, Experience)

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

box = sns.boxplot(x="sex", y="lwage", data=data ,palette='pastel')
plt.xlabel('Sexo')
plt.ylabel('Logaritmo del salario por hora')

# The real wage quartiles are increasing with the educational level.
# Lower salary dispersion for the postgraduate level. pastel

The average logarithm of salary is higher for men than for women

In [ ]:
data['wage'].plot(kind = 'hist', bins = 50, figsize = (8,6))
plt.title('Salario por hora')

txt="Elaboración propia"  
plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)
plt.xlim(0, 150)
plt.show()

The most common hourly wage is 20 and there is a distribution with a left tail. The highest salary is 130.

For the women, 

In [ ]:
sns.regplot(data=data, x="exp1", y="lwage", x_bins=np.arange(0, 40, 5), order=1)

The more years of experience, the people get a higher salary.

In [ ]:
sns.set_style("white")
gridobj = sns.lmplot(x="exp1", y="lwage", 
                     data=data, 
                     height=7, 
                     robust=True, 
                     palette='Set1', 
                     col="sex",
                     scatter_kws=dict(s=60, linewidths=0.7, edgecolors='black'))

gridobj.set(xlim=(0, 80), ylim=(0, 10))
plt.show()

At first glance, it is seen that experience has a positive impact on salary. But, in women this effect is smaller than in men due to persistent gender gaps in the labor market.

Plot the confidence Interval of sex's coefficient for a different models (basic, control, and partially out). All three coefficients must be in one figure. Explain what you find.

In [ ]:
coef_df = pd.DataFrame({
    'coef': [nocontrol_est, control_est, partial_est],
    'err': [nocontrol_se2, control_se, partial_se],
    'varname': ['No control', 'Con controles', 'Parcialmente fuera']
})

# Reorganizar las columnas
coef_df = coef_df[['coef', 'err', 'varname']]

# Mostrar el dataframe
print(coef_df)

fig, ax = plt.subplots(figsize=(8, 3))
coef_df.plot(x='varname', y='coef', kind='bar', 
             ax=ax, color='none', 
             yerr='err', legend=False)
ax.set_ylabel('')
ax.set_xlabel('')
ax.scatter(x=pd.np.arange(coef_df.shape[0]), 
           marker='s', s=120, 
           y=coef_df['coef'], color='black')
ax.axhline(y=0, linestyle='--', color='black', linewidth=4)
ax.xaxis.set_ticks_position('none')
_ = ax.set_xticklabels(['No control', 'Con controles', 'Parcialmente fuera'], 
                       rotation=0, fontsize=16)


The coefficients are larger for the With Controls and Partially Out models, when the sample is restricted to people with advanced education. Likewise, the error is minimized when we use the last two models, so the confidence interval is smaller.

You will also include a replication of the next figure for both groups, female and male.You will have only two plotted lines (Actual/Predicted(fitted)) for these College-educated workers. You have to create two separate figures, one for female and one for male. Could you explain the different patterns that you find?

The patterns found are that wages are higher for people who attend college than for those who attend only high school. Then, the relationship between years of experience and salary is only positive, both for university and high school, until before reaching 35 years of experience.

In [ ]:

data_hsg = data[data['hsg'] == 1]
data_clg = data[data['clg'] == 1]
data_scl = data[data['scl'] == 1]

data_clgm = data_clg[data_clg['sex'] == 0]  # Hombres
data_clgf = data_clg[data_clg['sex'] == 1]  # Mujeres



import pandas as pd

# Table_hsg
Tabla_hsg = data_hsg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

nivel_hsg = sorted(data_hsg['exp2'].unique())

Promedio = []
for nivel in nivel_hsg:
    Promedio.append(data_hsg[data_hsg['exp2'] <= nivel]['lwage'].mean())

Tabla_hsg['PromMov'] = Promedio
print(Tabla_hsg.head())

# Table_clg
Tabla_clg = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

Tabla_clgm = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()
Tabla_clgf = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

nivel_clg = sorted(data_clg['exp2'].unique())
nivel_clgm = sorted(data_clgm['exp2'].unique())
nivel_clgf = sorted(data_clgf['exp2'].unique())

Promedio = []
for nivel in nivel_clg:
    Promedio.append(data_clg[data_clg['exp2'] <= nivel]['lwage'].mean())

Tabla_clg['PromMov'] = Promedio
Tabla_clgm['PromMov'] = Promedio
Tabla_clgf['PromMov'] = Promedio
print(Tabla_clg.head())

import statsmodels.api as sm
import pandas as pd

formula = 'lwage ~ sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'

control_fit1 = sm.formula.ols(formula, data=data).fit()

predict = control_fit1.predict(data)

data['Predict'] = predict


data_sclP = data[data['scl'] == 1]
data_clgP = data[data['clg'] == 1]
data_hsgP = data[data['hsg'] == 1]

data_clgPm = data_clgP[data_clgP['sex'] == 0]  # Hombres
data_clgPf = data_clgP[data_clgP['sex'] == 1]  # Mujeres

import pandas as pd
###########################################################

Tabla_hsgP = data_hsgP.groupby('exp2')['Predict'].mean().reset_index()

nivel_hsgP = sorted(data_hsgP['exp2'].unique())

Promedio = []
for nivel in nivel_hsgP:
    Promedio.append(data_hsgP[data_hsgP['exp2'] <= nivel]['Predict'].mean())

Tabla_hsgP['PromMovP'] = Promedio
print(Tabla_hsgP.head())

Tabla_clgP = data_clgP.groupby('exp2')['Predict'].mean().reset_index()
Tabla_clgPf = data_clgPf.groupby('exp2')['Predict'].mean().reset_index()
Tabla_clgPm = data_clgPm.groupby('exp2')['Predict'].mean().reset_index()

nivel_clgP = sorted(data_clgP['exp2'].unique())

Promedio = []
for nivel in nivel_clgP:
    Promedio.append(data_clgP[data_clgP['exp2'] <= nivel]['Predict'].mean())

Tabla_clgP['PromMov'] = Promedio
Tabla_clgPf['PromMov'] = Promedio

Promediof = Promedio[:-1]
Tabla_clgPm['PromMov'] = Promediof
print(Tabla_clgP.head())

import matplotlib.pyplot as plt

x = Tabla_clg['exp2']
x_3 = Tabla_clgPm['exp2']
y = Tabla_clg['PromMov']
y_3 = Tabla_clgPm['PromMov']

# Create the graphic
plt.plot(x, y, color='navy', linestyle='-', label="Actual CLG")

plt.plot(x_3, y_3, color='darkred', linestyle='--')

plt.ylim(3, 3.2)
plt.xlim(0, 15)
plt.xlabel("Years of Potential Experience")
plt.ylabel("Log Wage (or Wage Gap)")
plt.title("Figure 10: Comparison between actual and fitted for CLG and HSG Male")
plt.grid(linestyle='--', color='gray')

plt.xticks(range(0, 16, 5))

plt.legend(loc="upper right", fontsize=8)

plt.show()

x = Tabla_clg['exp2']
x_3 = Tabla_clgPf['exp2']
y = Tabla_clg['PromMov']
y_3 = Tabla_clgPf['PromMov']

# Create the graphic
plt.plot(x, y, color='navy', linestyle='-', label="Actual CLG")

plt.plot(x_3, y_3, color='darkred', linestyle='--')

plt.ylim(3, 3.2)
plt.xlim(0, 15)
plt.xlabel("Years of Potential Experience")
plt.ylabel("Log Wage (or Wage Gap)")
plt.title("Figure 10: Comparison between actual and fitted for CLG and HSG Female")
plt.grid(linestyle='--', color='gray')

plt.xticks(range(0, 16, 5))

plt.legend(loc="upper right", fontsize=8)

plt.show()

The patterns found are that wages are higher for people who attend college than for those who attend only high school. Then, the relationship between years of experience and salary is only positive, both for university and high school, until before reaching 35 years of experience. Then, if we differentiate between sex the difference is very small.